In [4]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [5]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [6]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'content': '開催中 2025年3月20日(祝)～6月3日(火) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都) CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ) *   [美術展・博物展](https://www.walkerplus.com/event_list/eg0107/) 開催中 2025年3月20日(祝)～6月3日(火) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都) CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ) *   [グルメ・フードフェス](https://www.walkerplus.com/event_list/eg0117/) *   ![Image 6](https://www.walkerplus.com/asset/images/common/ico_charge.svg) 終了間近 2025年4月1日(火)～5月31日(土) 東京駅(東京都), 二重橋前駅(東京都), 京橋駅(東京都), 大手町駅(東京都), 有楽町駅(東京都) *   [グルメ・フードフェス](https://www.walkerplus.com/event_list/eg0117/) *   ![Image 8](https://www.walkerplus.com/asset/images/common/ico_parking.svg) 2025年4月5日(土)～6月15日(日) 東京駅(東京都), 二重橋前駅(東京都), 大手町駅(東京都), 日本橋駅(東京都), 京橋駅(東京都) *   [美術展・博物展](https://www.walkerplus.com/event_list/eg0107/) 2025年3月1日(土)～6月1日(日) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 東京駅(

In [7]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [8]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [9]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [10]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [11]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

具体的には、東京都の面積は約2,194.05 km²であるのに対し、沖縄県の面積は約2,281 km²です。従って、面積では沖縄県が東京都よりも約87 km²広いことになります。


In [12]:
# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

最近1ヶ月以内に開催される東京駅のイベントについて、いくつかの情報をお伝えします。

1. **東京駅イベント カレンダー**  
   - **開催日**: 2025年8月5日〜2025年8月31日  
   - **詳細**: 東京駅のイベントに関する情報が掲載されています。公式サイトなどで確認できます。  
   - [詳しくはこちら](https://marunouchi.jp-kitte.jp/event/calendar.jsp)

2. **東京都市 駅ピポコレンド特集**  
   - **内容**: 東京駅周辺の飲食店やイベント情報が掲載されています。特定のイベントは月に1回開催されています。  
   - [詳しくはこちら](https://www.jalan.net/kankou/spt_guide000000204974/event/)

3. **新着情報 (GranSta)**  
   - **内容**: GranSta 東京駅での新しいショップやレストランのイベント情報。  
   - [詳しくはこちら](https://www.gransta.jp/news/)

4. **東京駅POPUP**  
   - **開催日**: 6月5日〜6月24日  
   - **詳細**: 東京駅にてPOPUPイベントが開催されている。KAWANOWAの「幾の躍」での開催情報。  
   - [詳しくはこちら](https://www.kawanowa.com/popup_tokyostation_june/)

5. **東京駅の団体観光イベント**  
   - **内容**: 大俵祭（定期開催）や多様な特別イベントが行われる。詳細なスケジュールはサイトをチェック。  
   - [詳しくはこちら](https://www.jalan.net/kankou/spt_guide000000204974/event/)

これらの情報を参考にして、興味のあるイベントに参加してみてください。詳しい内容については各リンクをご参照くださいますようお願いいたします。


In [13]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

'質問:日本で2番目に高い山を教えてください。'

日本で2番目に高い山は「北岳（きただけ）で、高さは3193メートルです。場所は山梨県にあり、最も高い山である富士山（3776メートル）の次に位置しています。北岳は、ハイキングや登山の人気スポットとなっています。


'質問:そこへ行く方法を教えてください。'

「そこへ行く方法」を知りたい場合、以下のリソースが役立ちます。

1. [Native Phrases Blog](https://native-phrase-blog.com/4628/) - そこの行き方に関する情報や，バスや電車などの交通手段について紹介しています。
   
2. [DMM英会話](https://eikaiwa.dmm.com/uknow/questions/129990/) - 行き方に関する英語表現を学びたい方におすすめのページです。
   
3. [Reverso Context](https://context.reverso.net/%E7%BF%BB%E8%A8%B3/%E6%97%A5%E6%9C%AC%E8%AA%9E-%E8%8B%B1%E8%AA%9E/%E3%81%9D%E3%81%93%E3%81%B8%E8%A1%8C%E3%81%8F) - 行き方に関する例文や翻訳が見つかります。
   
これらのリンクを参考に、具体的な行き方や使うべき交通機関についての情報を得ることができます。どのような手段で行くかは目的地や出発地点によりますので、具体的な場所を教えていただければ、さらに詳しいアドバイスができるかもしれません。

---ご利用ありがとうございました！---
